# Clustering Crypto

In [60]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [61]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [62]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [63]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path)
df = df.set_index('Unnamed: 0')
df.index.name = None
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [64]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df = df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [65]:
# Keep only cryptocurrencies that are trading
df = df[(df['IsTrading']==True)]
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [66]:
# Keep only cryptocurrencies with a working algorithm
## Assuming that a 'NaN' in the 'TotalCoinsMined' column is indicative of a broken algorithm
df = df.dropna()
df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [67]:
# Remove the "IsTrading" column
df = df.drop(columns='IsTrading',axis=1)
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [68]:
# Remove rows with at least 1 null value
df = df.dropna()
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [69]:
# Remove rows with cryptocurrencies having no coins mined
df = df[(df[['TotalCoinsMined']] != 0).all(axis=1)]
df.tail()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [70]:
# Drop rows where there are 'N/A' text values
df = df[~df['CoinName'].str.contains('N/A')]
df = df[~df['Algorithm'].str.contains('N/A')]
df = df[~df['ProofType'].str.contains('N/A')]
df.tail()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [71]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = df[['CoinName']]
coin_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [72]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = df.drop(columns=['CoinName'],axis=1)

crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [73]:
# Read 'TotalCoinSupply' as a float rather than an object for subsequent encoding
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype(float)

In [74]:
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [75]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# Standardize data
## Initialize standard scaler
scaler = StandardScaler()
X_scaler = scaler.fit(X)

## Scale the X data
X_scaled = X_scaler.transform(X)

### Reducing Dimensions Using PCA

In [77]:
# Use PCA to reduce dimensions to 3 principal components
## Initialize the PCA model with 3 principal components
pca = PCA(n_components=3)

# Get THREE principal components for the data.
X_pca = pca.fit_transform(X_scaled)


In [78]:
# Create a DataFrame with the principal components data
df_X_pca = pd.DataFrame(
    data=X_pca, columns=["principal component 1", "principal component 2",'principal component 3']
)
# Set Index to the Index above
df_X_pca = df_X_pca.set_index(crypto_df.index)
df_X_pca.head()


,principal component 1,principal component 2,principal component 3
42,-0.364161,0.946484,-0.580625
404,-0.347493,0.946548,-0.580880
1337,2.301515,1.553591,-0.640450
BTC,-0.132819,-1.298181,0.183405
ETH,-0.131478,-1.982332,0.395681


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [79]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_X_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [81]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_X_pca)
# Predict clusters
predictions = model.predict(df_X_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_df.join(df_X_pca)
clustered_df['CoinName'] = coin_names['CoinName']
clustered_df['Class'] = predictions
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.364161,0.946484,-0.580625,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.347493,0.946548,-0.580880,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.301515,1.553591,-0.640450,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.132819,-1.298181,0.183405,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.131478,-1.982332,0.395681,Ethereum,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [82]:
# Create a 3D-Scatter with the PCA data and the clusters
from mpl_toolkits.mplot3d import Axes3D
fig = px.scatter_3d(clustered_df, x = 'principal component 1',  
                    y = 'principal component 2',  
                    z = 'principal component 3', 
                    color = 'Class',
                   hover_name="CoinName",
                   hover_data=["Algorithm"]) 
fig.show()

#### Table of Tradable Cryptocurrencies

In [83]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"],
               sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [84]:
# Print the total number of tradable cryptocurrencies
## Count the number of unique values in 'CoinName' column
tradable_crypto = clustered_df['CoinName'].nunique()
print(f'The total number of tradable cryptocurrencies is, currently, {tradable_crypto}.')

The total number of tradable cryptocurrencies is, currently, 532.


#### Scatter Plot with Tradable Cryptocurrencies

In [100]:
# Scale data to create the scatter plot
X = clustered_df['TotalCoinsMined'].values.reshape(-1,1)
y = clustered_df['TotalCoinSupply'].values.reshape(-1,1)

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_df = pd.DataFrame(X,columns=['TotalCoinsMined'])
Y_df = pd.DataFrame(y,columns=['TotalCoinSupply'])
#coin_name =clustered_df['CoinName']

clustered_scaled = X_df.join(Y_df)
clustered_scaled = clustered_scaled.set_index(clustered_df.index)
clustered_scaled['CoinName'] = clustered_scaled.index
clustered_scaled = clustered_scaled.reset_index()
clustered_scaled = clustered_scaled.drop(columns='index',axis=1)
clustered_scaled.head()

,TotalCoinsMined,TotalCoinSupply,CoinName
0,-0.116748,-0.152865,42
1,-0.093589,-0.144996,404
2,0.525872,4.493764,1337
3,-0.116354,-0.152554,BTC
4,-0.114384,-0.152865,ETH


In [102]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_scaled.hvplot.scatter(x='TotalCoinsMined',y='TotalCoinSupply',by='CoinName',hover_cols=['CoinName'])



:NdOverlay   [CoinName]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)